# **Data Collection and processing**

In [4]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

print('started')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
df_combined = pd.read_csv('/kaggle/input/preprocessed-post-csv/preprocessed_post.csv')

df_combined.dropna(subset=['processed_post'], inplace=True)

started
/kaggle/input/preprocessed-post-csv/preprocessed_post.csv


# **Embedding**

In [5]:
import gensim

sentences = [post.split() for post in df_combined['processed_post']]
print('sentences',len(sentences))
word2vec_model = gensim.models.Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)

vocab_size = len(word2vec_model.wv)
print('123456',vocab_size)

# def embed_sentence_average(sentence, word2vec_model):
#     embeddings = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
   
#     if len(embeddings) == 0:
#         return np.zeros(300)  # Return a zero vector if no embeddings are found
#     return np.mean(embeddings, axis=0)  
#     #return np.array([word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(300) for word in sentence])

# # import numpy as np
# # embeded_post= [embed_sentence(post.split(), word2vec_model) for post in df_combined['processed_post']]

# X_features = np.array([embed_sentence_average(post.split(), word2vec_model) for post in df_combined['processed_post']])
# print('X featured', X_features[:2], len(X_features))





sentences 101357
123456 100486


# **Padding**

In [6]:
def embed_sentence(sentence, word2vec_model):
    return np.array([word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(300) for word in sentence])

import numpy as np
embeded_post= [embed_sentence(post.split(), word2vec_model) for post in df_combined['processed_post']]
from keras.preprocessing.sequence import pad_sequences
#X_padded = pad_sequences([x for x in embeded_post], maxlen=300, dtype='float32', padding='post')
maxlen = 100

# Pad sequences with maxlen
X_padded = pad_sequences([x for x in embeded_post], maxlen=maxlen, dtype='float32', padding='post')

In [9]:
X_features.shape


(101357, 300)

In [3]:
df_combined['label'].shape

NameError: name 'df_combined' is not defined

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, df_combined['label'], test_size=0.2, random_state=42, stratify=df_combined['label'])

In [6]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
print(X_train.shape, X_test.shape,'test train split')
print(np.bincount(y_train),np.bincount(y_test), ' ration of 0:1')

(81085, 300) (20272, 300) test train split
[70601 10484] [17651  2621]  ration of 0:1


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Create the RandomForestClassifier
print("model started")
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
print("fitting started")
# Fit the model on your training data
rf_clf.fit(X_train, y_train)
print("prediction started")
# Predict on test data
y_pred_rf = rf_clf.predict(X_test)

# Print classification report
print("Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))

model started
fitting started
prediction started
Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     17651
           1       0.86      0.69      0.77      2621

    accuracy                           0.95     20272
   macro avg       0.91      0.84      0.87     20272
weighted avg       0.94      0.95      0.94     20272



In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Create the MultinomialNB model with alpha parameter (smoothing)
nb_clf = MultinomialNB(alpha=1.0)

# Fit the model on your training data
nb_clf.fit(X_train, y_train)

# Predict on test data
y_pred_nb = nb_clf.predict(X_test)

# Print classification report
print("Multinomial Naive Bayes Classifier:")
print(classification_report(y_test, y_pred_nb))


ValueError: Negative values in data passed to MultinomialNB (input X)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Use TfidfVectorizer to convert text data to TF-IDF representation
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df_combined['processed_post'])



In [19]:
from sklearn.model_selection import train_test_split
X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X_tfidf, df_combined['label'], test_size=0.2, random_state=42, stratify=df_combined['label'])

In [1]:
print(X_train_nb.shape, X_test_nb.shape,'test train split')
print(np.bincount(y_train_nb),np.bincount(y_test_nb), ' ration of 0:1')

NameError: name 'X_train_nb' is not defined

In [22]:
# Train a MultinomialNB model
nb_clf = MultinomialNB(alpha=1.0)
nb_clf.fit(X_train_nb, y_train_nb)
y_pred_nb = nb_clf.predict(X_test_nb)

# Print classification report
print("Multinomial Naive Bayes Classifier:")
print(classification_report(y_test_nb, y_pred_nb))

Multinomial Naive Bayes Classifier:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     17651
           1       0.91      0.48      0.63      2621

    accuracy                           0.93     20272
   macro avg       0.92      0.73      0.79     20272
weighted avg       0.92      0.93      0.92     20272



In [8]:
from sklearn.model_selection import train_test_split
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X_padded, df_combined['label'], test_size=0.2, random_state=42, stratify=df_combined['label'])

In [10]:
y_train_nn = np.array(y_train_nn)
y_test_nn = np.array(y_test_nn)

In [12]:
print(X_train_nn.shape, X_test_nn.shape,'test train split')
print(np.bincount(y_train_nn),np.bincount(y_test_nn), ' ration of 0:1')

(81085, 100, 300) (20272, 100, 300) test train split
[70601 10484] [17651  2621]  ration of 0:1


In [13]:
from sklearn.utils import class_weight

# Assuming y_train contains your labels (0 for non-suicidal, 1 for suicidal)
class_weights = class_weight.compute_class_weight('balanced', classes=[0, 1], y=y_train_nn)

# Create a dictionary for class weights
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(class_weight_dict)

{0: 0.5742482401099135, 1: 3.867083174360931}


In [ ]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, Input

# Assuming you have defined y_train (your target labels)
x2_model = Sequential([
        Input(shape=(X_train_nn.shape[1], X_train_nn.shape[2])),
    GRU(128, return_sequences=True),
    Dropout(0.2),
    GRU(64),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

x2_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
x2_model.fit(X_train_nn, y_train_nn, epochs=10, batch_size=32, class_weight=class_weight_dict)